In [1]:
import pandas as pd
from pythainlp import word_tokenize as tokenize
from pythainlp.util import *
from tqdm.auto import tqdm
import os
import sys
sys.path.append('../Cython')
from Method import *
tqdm.pandas()

C:\Users\Alps\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
path = "../assets/CleanedData"
agent = os.listdir(path)
# print(agent)

In [3]:
df = pd.DataFrame()
date = "10_10_2021"

for i in agent:
    # print(path + "/" + i + "/" + date + ".csv")
    try:
        df_temp = pd.read_csv(path + "/" + i + "/" + date + ".csv")
        df = df.append(df_temp, ignore_index=True)
        # print("success")
    except:
        pass

df

,title,summary,link,published
0,“มิสยูนิเวิร์สไทยแลนด์ 2021” พร้อมใจกันตักบาตร...,กองประกวด “มิสยูนิเวิร์สไทยแลนด์ 2021” (Miss U...,https://bangkok-today.com/%e0%b8%a1%e0%b8%b4%e...,10_10_2021
1,อดีตในการ “ปราบม็อบ”..มักชอบอ้างมีอาวุธ??,“ร้องไห้คร่ำครวญกลางที่ประชุมครม.” “อดีตรัฐมนต...,https://bangkok-today.com/%e0%b8%ad%e0%b8%94%e...,10_10_2021
2,ร็อคกิ้งคิดส์ฉลองเปิด Rocking Kids Store ศูนย์...,แบรนด์ Rocking Kids (ร็อคกิ้งคิดส์) จุดพลุฉลอง...,https://bangkok-today.com/%e0%b8%a3%e0%b9%87%e...,10_10_2021
3,โควิดดันราคารถมือสองไทยปรับขึ้นชั่วคราว จับตาส...,การแพร่ระบาดของโควิดไปทั่วโลก แม้ด้านหนึ่งจะส่...,https://bangkok-today.com/%e0%b9%82%e0%b8%84%e...,10_10_2021
4,โรงพยาบาลเอกชนปี’ 64 คาดรายได้ฟื้นตัวจากฐานที่...,การระบาดของโควิดที่รุนแรงขึ้นโดยเฉพาะในช่วงเมษ...,https://bangkok-today.com/%e0%b9%82%e0%b8%a3%e...,10_10_2021
...,...,...,...,...
573,&quot;ซานโตส&quot; จวก &quot;โซลชาร์&quot; ใช้...,โซลชาร์ โดนด่าอีกแล้ว หลังกุนซือทีมชาติโปรตุเก...,https://www.thairath.co.th/sport/eurofootball/...,10_10_2021
574,บุหรี่ขาดตลาด สรรพสามิต เร่งหาต้นตอปัญหา คนเริ...,บุหรี่ขาดตลาด หลังปรับโครงสร้างภาษีใหม่ ขณะที่...,https://www.thairath.co.th/business/economics/...,10_10_2021
575,เกือบเนียน แฟนบอลปารากวัย สวมรอยเป็นนักเตะก่อน...,แฟนบอลปารากวัย ตีเนียนสวมรอยเป็นนักเตะทีมชาติ ...,https://www.thairath.co.th/sport/worldcup/2215752,10_10_2021
576,เผยคนร้องเรียนถูกหลอกขายของออนไลน์จากเฟซบุ๊กมา...,รัฐบาลเร่งดำเนินคดีฉ้อโกง หลอกขายของออนไลน์ พบ...,https://www.thairath.co.th/news/politic/2215811,10_10_2021


In [4]:
NEWMM = "newmm"
ATTACUT = "attacut"
list_column = df.columns.values.tolist()
column_name = list_column[1]

def word_break(news,engine):
    segment = tokenize(news,keep_whitespace=False,engine=engine)
    return segment

In [5]:
atta_list = []
newmm_list = []

def wordBreak(df):
    l = len(df)
    for i in tqdm(range(l)):
        atta_list.append(word_break(df.iloc[i][column_name],ATTACUT))
        newmm_list.append(word_break(df.iloc[i][column_name],NEWMM))

wordBreak(df)

100%|██████████| 578/578 [00:26<00:00, 21.52it/s]


In [6]:
def LCS(atta_list, newmm_list):
    l = len(atta_list)
    lcs_atta = set()
    lcs_newmm = set()
    for i in tqdm(range(l-1)):
        w1_atta = atta_list[i]
        len_w1_atta = len(w1_atta)
        w1_newmm = newmm_list[i]
        len_w1_newmm = len(w1_newmm)
        for j in range(i+1,l):
            w2_atta = atta_list[j]
            len_w2_atta = len(w2_atta)
            w2_newmm = newmm_list[j]
            len_w2_newmm = len(w2_newmm)
            lcs_atta_list,maxl_atta = [],0
            lcs_newmm_list,maxl_newmm = [],0
            if i != j:
                lcs_atta_list,maxl_atta = lcs(w1_atta,w2_atta,len_w1_atta,len_w2_atta)
                lcs_newmm_list,maxl_newmm = lcs(w1_newmm,w2_newmm,len_w1_newmm,len_w2_newmm)
                if lcs_atta_list != []:
                    lcs_atta.update(lcs_atta_list)
                if lcs_newmm_list != []:
                    lcs_newmm.update(lcs_newmm_list)
    return list(lcs_atta), list(lcs_newmm)

In [7]:
def count_trend(trend_list,df):
    count_list = []
    frequency_list = []
    for i in tqdm(trend_list):
        count = df[column_name].str.contains(str(i), case=False, regex=False).sum()
        frequency = sum(str(x).count(i) for x in df[column_name])
        count_list.append(count)
        frequency_list.append(frequency)
    
    result = pd.DataFrame({'word': trend_list, 'count': count_list, 'frequency': frequency_list})
    result["n(news)"] = [len(df)] * len(result)
    result.sort_values(["frequency"], ascending=False, inplace=True, ignore_index=True)
    return result

In [8]:
lcs_atta, lcs_newmm = LCS(atta_list, newmm_list)

100%|██████████| 577/577 [00:40<00:00, 14.37it/s]


In [9]:
result = count_trend(list(lcs_newmm),df)
result.head(20)

100%|██████████| 1350/1350 [00:05<00:00, 250.60it/s]


,word,count,frequency,n(news)
0,โควิด-19,52,84,578
1,เตอร์,46,81,578
2,พล.อ.,21,30,578
3,พล.อ.ประ,20,28,578
4,ล.อ.ประยุทธ์,17,23,578
5,พล.อ.ประยุทธ์,17,23,578
6,โรคโควิด-19,13,22,578
7,ได้รับผลกระทบจาก,12,22,578
8,มีน้ำท่วมขัง,4,21,578
9,ขังในพื้นที่,2,19,578


In [10]:
result = count_trend(list(lcs_atta),df)
result.head(20)

100%|██████████| 2440/2440 [00:10<00:00, 225.62it/s]


,word,count,frequency,n(news)
0,[…],283,283,578
1,โควิด-19,52,84,578
2,ที่ผ่านมา,51,73,578
3,ผู้ติดเชื้อ,27,58,578
4,เมื่อวันที่,38,48,578
5,นอกจากนี้,22,36,578
6,ที่ได้รับ,20,36,578
7,ได้รับผล,18,35,578
8,รับผลกระทบ,18,34,578
9,ได้รับการ,24,33,578
